In [ ]:
pip install opencv-python mediapipe screen-brightness-control numpy

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import screen_brightness_control as sbc
import time
from IPython.display import display, clear_output
from PIL import Image
import winsound
import time

In [ ]:
# --- Initialize MediaPipe FaceMesh ---
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False)

In [ ]:
# --- Webcam feed ---


print("Press 'q' to quit.")
# Set frame count
frame_count = 0
MAX_FRAMES = 100  # Runs for ~100 frames (~5-6 seconds)

In [ ]:
# --- Define eye landmark indices---
LEFT_EYE = [33, 133, 160, 159, 158, 157, 173]
RIGHT_EYE = [362, 263, 387, 386, 385, 384, 398]

In [ ]:
# --- Lighting thresholds ---
CONTRAST_THRESHOLD = 30
GLARE_THRESHOLD = 120

In [ ]:
"""cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    h, w, _ = frame.shape
    warning = ""

    if results.multi_face_landmarks:
        lm = results.multi_face_landmarks[0]
        points = [(int(p.x * w), int(p.y * h)) for p in lm.landmark]

        def crop_eye(indices):
            pts = np.array([points[i] for i in indices])
            x, y, ww, hh = cv2.boundingRect(pts)
            # Clamp to image bounds
            x = max(x, 0)
            y = max(y, 0)
            if x + ww > frame.shape[1] or y + hh > frame.shape[0]:
                return None  # invalid crop
            
            return frame[y:y+hh, x:x+ww]

        left_eye = crop_eye(LEFT_EYE)
        right_eye = crop_eye(RIGHT_EYE)

        def glare_score(eye_img):
            gray = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
            return np.mean(gray)

# Check for valid eye crops before processing
        if (
            left_eye is not None and right_eye is not None and
            left_eye.size > 0 and right_eye.size > 0
        ):
            avg_glare = (glare_score(left_eye) + glare_score(right_eye)) / 2
            contrast = np.std(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
            
            msg = f"Contrast: {contrast:.1f} | Glare: {avg_glare:.1f}"
            cv2.putText(frame, msg, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    # Decision logic
            if contrast < CONTRAST_THRESHOLD:
                warning = "Ambient lighting too low!"
            elif avg_glare > GLARE_THRESHOLD:
                warning = "High screen glare!"
            if warning:
                cv2.putText(frame, warning, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        else:
    # Eye detection failed
            msg = "Cannot detect face or eyes!"
            cv2.putText(frame, msg, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        for i in LEFT_EYE + RIGHT_EYE:
            cv2.circle(frame, points[i], 2, (0, 255, 0), -1)

    cv2.imshow("Red Light Reader", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()"""

In [ ]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

last_warning = None
last_sound_time = 0
SOUND_COOLDOWN = 1 

def play_warning_sound():
    frequency = 500  # Hertz
    duration = 100    # milliseconds
    winsound.Beep(frequency, duration)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    h, w, _ = frame.shape
    warning = ""

    if results.multi_face_landmarks:
        lm = results.multi_face_landmarks[0]
        points = [(int(p.x * w), int(p.y * h)) for p in lm.landmark]

        def crop_eye(indices):
            pts = np.array([points[i] for i in indices])
            x, y, ww, hh = cv2.boundingRect(pts)
            # Clamp to image bounds
            x = max(x, 0)
            y = max(y, 0)
            if x + ww > frame.shape[1] or y + hh > frame.shape[0]:
                return None  # invalid crop
            
            return frame[y:y+hh, x:x+ww]

        left_eye = crop_eye(LEFT_EYE)
        right_eye = crop_eye(RIGHT_EYE)

        def glare_score(eye_img):
            gray = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
            return np.mean(gray)

# Check for valid eye crops before processing
        if (
            left_eye is not None and right_eye is not None and
            left_eye.size > 0 and right_eye.size > 0
        ):
            avg_glare = (glare_score(left_eye) + glare_score(right_eye)) / 2
            contrast = np.std(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
            
            msg = f"Contrast: {contrast:.1f} | Glare: {avg_glare:.1f}"
            cv2.putText(frame, msg, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    # Decision logic
            if contrast < CONTRAST_THRESHOLD:
                warning = "WARNING: Ambient lighting too low!"
            elif avg_glare > GLARE_THRESHOLD:
                warning = "WARNING: High screen glare!"
            else:
                warning = None
            
            if warning:
                current_time = time.time()
                
                if warning != last_warning or (current_time - last_sound_time) > SOUND_COOLDOWN:
                    play_warning_sound()
                    last_sound_time = current_time
                    last_warning = warning
                    
                cv2.rectangle(frame, (5,60), (500,90),(0,0,0), -1)
                cv2.putText(frame, warning, (10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                
                play_warning_sound()
            
            else:
                last_warning = None
        
        else:
    # Eye detection failed
            msg = "WARNING: Unable to detect face or eyes!"
            cv2.putText(frame, msg, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        for i in LEFT_EYE + RIGHT_EYE:
            cv2.circle(frame, points[i], 2, (0, 255, 0), -1)

    cv2.imshow("Red Light Reader", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()